In [1]:
import numpy as np
from numpy import random

from matplotlib import pyplot as plt
from numpy import linalg as la
from enum import Enum

In [2]:
class ProgressBar(object):
    
    def __init__(self, total, prefix='', suffix='', decimals=1, length=100, fill='█'):
        """
        Creates terminal progress bar instance.
        @params:
            total      - Required  : total iterations (Int)
            prefix     - Optional  : prefix string (Str)
            suffix     - Optional  : suffix string (Str)
            decimal    - Optional  : positive number of decimals in percent complete (Int)
            length     - Optional  : character length of bar (Int):
            fill       - Optional  : bar fill character (Str)
        """
        self.total = total
        self.prefix = prefix
        self.suffix = suffix
        self.decimals = decimals
        self.length = length
        self.fill = fill


    def print_bar(self, iteration):
        """
        Updates progress bar status.
        @params:
            iteration  - Required  : current iteration (Int)
        """
        percent = ("{0:." + str(self.decimals) + "f}").format(
                        100 * (iteration / float(self.total)))
        filled_length = int(self.length * iteration // self.total)
        bar = self.fill * filled_length + '-' * (self.length - filled_length)

        print('\r{} |{}| {}% {}'.format(self.prefix, bar, percent, self.suffix), end='\r')

        # print a new line on complete
        if iteration == self.total:
            print()


# def test_progress_bar(n_items=50, length=50, sleep_interval=.2):
    
#     from time import sleep

#     pb = ProgressBar(n_items, prefix='Progress:', suffix='Complete', length=length)
#     for i in range(0, n_items):
#         sleep(sleep_interval)
#         pb.print_bar(i)

#     pb.print_bar(n_items)

# test_progress_bar(57)

In [3]:
def normalize(x):
    x /= la.norm(x)
    return x


class Shape(object):
    
    def get_color(self, surface_point=None):
        pass

    def get_normal(self, surface_point):
        pass
    
    def intersect(self, start, direction):
        return np.inf


class Sphere(Shape):
    
    def __init__(self, center, radius, surface_color, reflection=0, 
                 diffuse_color=1., specular_color=1., transparency=0, ior=1.1):
        
        self.center = center
        self.radius = radius
        self.surface_color = surface_color
        self.reflection = reflection
        self.diffuse_color = diffuse_color
        self.specular_color = specular_color
        self.transparency = transparency
        self.ior = ior
        
            
    def get_color(self, surface_point=None):
        """
        Return a color of a distinct ``surface_point``. If the sphere has
        only one color the ``surface_point`` affects nothing and can be 
        omitted.
        @params:
            surface_point  : a distinct point on the sphere surface that 
                             color is required. 
        """
        if hasattr(self.surface_color, '__len__'):
            return self.surface_color
        else:
            return self.surface_color(surface_point)
    
    
    def get_normal(self, surface_point):
        return normalize(surface_point - self.center)
    
    
    def intersect(self, start, direction):
        
        v = self.center - start
        vd = np.dot(direction, v)
        v_r = np.dot(v, v) - self.radius ** 2
        disc = vd ** 2 - v_r

        if disc > 0:
            root = np.sqrt(disc)
            t0, t1 = vd - root, vd + root
            if t0 > 0:
                return t0
            elif t1 > 0:
                return t1

        return np.inf

    
class Plane(Shape):
    
    def __init__(self, init_point, normal, surface_color, reflection=0, diffuse_color=1., specular_color=1., transparency=0):
        self.init_point = init_point
        self.normal = normal
        self.surface_color = surface_color
        self.reflection = reflection
        self.diffuse_color = diffuse_color
        self.specular_color = specular_color
        self.transparency = transparency

    
    def get_color(self, surface_point=None):
        """
        Return a color of a distinct ``surface_point``. If the plane has
        only one color the ``surface_point`` affects nothing and can be 
        omitted.
        @params:
            surface_point  : a distinct point on the sphere surface that 
                             color is required. 
        """
        if hasattr(self.surface_color, '__len__'):
            return self.surface_color
        else:
            return self.surface_color(surface_point)
        
    
    def get_normal(self, surface_point):
        return self.normal
    
        
    def intersect(self, start, direction):
    
        denom = np.dot(direction, self.normal)
        if np.abs(denom) < 1e-6:
            return np.inf

        d = np.dot(self.init_point - start, self.normal) / denom
        if d < 0:
            return np.inf

        return d

In [4]:
class Id(object):
    """
    ID generator - a singleton class.

    Example:
        ID = Id()
        Id.get()
    """

    class __Id:

        def __init__(self):
            self.generator = self.id_gen()

        def id_gen(self):
            id_ = 0
            while True:
                yield id_
                id_ += 1

        def get(self):
            return next(self.generator)
        
        def reset(self):
            self.generator = self.id_gen()

    instance = None

    def __new__(cls):
        if not Id.instance:
            Id.instance = Id.__Id()
        return Id.instance

    def __getattr__(self, name):
        return getattr(self.instance, name)

    def __setattr__(self, name):
        return setattr(self.instance, name)


class Ray(object):
    
    class Type(Enum):
        CAMERA = 0
        REFLECTED = 1
        REFRACTED = 2
    
    def __init__(self, start, direction, att=1., cum_att=1., inside=False, type_=Type.CAMERA):
        """
        Creates a new instance of ``Ray``.
        @params:
            ior     : an index of refraction of the media
            inside  : show whether ray is inside of the shape or not .
        """
        self.id = Id().get()
        self.type = type_
        
        # geometrical properties
        self.start = start
        self.direction = direction
        self.end = None
        self.length = -1
        
        # Propagation parameters including transmission media change
        self.att = att
        self.cum_att = cum_att
        self.inside = inside
    
    def __str__(self):
        string = '<{}({}) start={} end={} dir={} len={:.4}; att={:.4}({:.4}) ior={:.4}>'
        return string.format(self.type.name, self.id, self.start, self.end, 
                             self.direction, self.length, self.att,
                             self.cum_att, self.inside)


class RayNode(object):
    
    def __init__(self, ray, parent=None, tree=None):
        
        self.ray = ray
        self.parent = parent
        self.reflected = None
        self.refracted = None
        self.tree=None
        
        self.id = ray.id
        self.depth = parent.depth + 1 if parent else 0
    
    def __str__(self):
        reflected_id = str(self.reflected.ray.id) if self.reflected else '.'
        refracted_id = str(self.refracted.ray.id) if self.refracted else '.'
        self_id = str(self.ray.id)
        return '({})<({})>({})'.format(refracted_id, self_id, reflected_id)
    
    def add_child(self, ray):
        node = RayNode(ray, parent=self, tree=tree)
        
        if ray.type is Ray.Type.REFLECTED:
            self.reflected = node
        elif ray.type is Ray.Type.REFRACTED:
            self.refracted = node

    def view(self):
        refracted_view = self.refracted.view() if self.refracted else '.'
        reflected_view = self.reflected.view() if self.reflected else '.'
        return '[{}]<({})>[{}]'.format(refracted_view, str(self.ray.id), reflected_view)


class RayTree(object):
    
    def __init__(self, camera_ray):
        self.root = RayNode(camera_ray, tree=self)
        self.rays = dict((camera_ray.id, camera_ray))
        self.levels = dict((0, camera_ray))
    
    def __str__(self):
        return camera_ray.view()


In [5]:
BG_COLOR = np.array([0., 0., 0.3])

class RayTracer(object):
    
    def __init__(self, scene, light_point, color_light, cam_point, ambient, specular_k):
        self.scene = scene
        self.light_point = light_point
        self.color_light = color_light
        self.cam_point = cam_point
        self.ambient = ambient
        self.specular_k = specular_k

        
    def trace_ray(self, ray_start, ray_dir):
        """
        The method computes find nearest intersected objects, compute ray
        parameters and color.
        """
        # Find the nearest intersection point with the scene.
        t = np.inf
        for s in self.scene:
            t_i = s.intersect(ray_start, ray_dir)
            if t_i < t:
                t, shape = t_i, s

        # Return None if the ray does not intersect any object.
        if t == np.inf:
            return

        # Find the intersection point on the object and properties of the object.
        hit_point = ray_start + ray_dir * t

        normal = shape.get_normal(hit_point)
        color = shape.get_color(hit_point)

        to_light = normalize(self.light_point - hit_point)
        to_cam = normalize(self.cam_point - hit_point)

        # Shadow: find if the point is shadowed or not.
        bias = normal * .0001
        l = [s.intersect(hit_point + bias, to_light) for s in self.scene if s != shape]
        if l and min(l) < np.inf:
            return

        # Start computing the color.
        ray_color = self.ambient

        # Lambert shading (diffuse).
        ray_color += shape.diffuse_color * max(np.dot(normal, to_light), 0) * color

        # Blinn-Phong shading (specular).
        ray_color += shape.specular_color * (max(np.dot(normal, normalize(to_light + to_cam)), 0) 
                                             ** self.specular_k * self.color_light)

        return shape, hit_point, normal, ray_color


    def trace_path(self, ray_start, ray_dir):
    
        color = np.zeros(3)
        
        # Define a list of rays to be traced to enable tracing both reflecting and refracting rays.
        # Thus tracing procedures perform width-first traversal.
        
        primary_ray = Ray(start=ray_start, direction=ray_dir)
        rays_to_trace = [(primary_ray, 0)]

        while rays_to_trace:

            ray, depth = rays_to_trace.pop(0)
            # If ray tracing reaches depth limit stop tracing this branch.
            if depth == DEPTH_MAX:
                continue

            traced = self.trace_ray(ray.start, ray.direction)
            if not traced:
                color += BG_COLOR
                continue

            shape, hit_point, normal, ray_color = traced
            bias = normal * .0001
            neg_dn = -np.dot(ray.direction, normal)

            # Reflection: create a new ray and append it to the rays list to trace further.
            start = hit_point + bias
            direction   = ray.direction + 2 * neg_dn * normal
            
            reflected_ray = Ray(start=start, direction=direction, att=shape.reflection,
                                cum_att = ray.cum_att * shape.reflection,
                                inside=ray.inside, type_=Ray.Type.REFLECTED)
            
            rays_to_trace.append((reflected_ray, depth + 1))

            # Refraction: create a new ray and append it to the rays list to trace further.
            if shape.transparency:

                eta = shape.ior if ray.inside else 1 / shape.ior
                k = 1 - eta ** 2 * (1 - neg_dn ** 2)

                if k >= 0: # total reflection under refraction into the environment with less ior

                    start = hit_point - bias
                    direction = ray.direction * eta + normal * (eta * neg_dn - np.sqrt(k))
                    
                    refracted_ray = Ray(start=start, direction=direction, att=shape.transparency,
                                        cum_att = ray.cum_att * shape.transparency,
                                        inside=not ray.inside, type_=Ray.Type.REFRACTED)

                    rays_to_trace.append((refracted_ray, depth + 1))

            
            fresnel_effect = .1 + (1-.1) * (1 - neg_dn) ** 3
            if ray.type == Ray.Type.REFLECTED:
                pass
            elif ray.type == Ray.Type.REFRACTED:
                fresnel_effect = 1 - fresnel_effect
            else:
                fresnel_effect = 1
                
            color += ray.cum_att * ray_color
            
            # Compute current color.
#             color += ray.cum_att * ray_color

        return color

In [6]:
class Render(object):
    
    def __init__(self, scene, light_point, color_light, 
                 specular_k, ambient, cam_point,
                 screen, width, height):
        
        self.scene = scene
        self.light_point = light_point
        self.color_light = color_light
        self.specular_k = specular_k
        self.ambient = ambient
        self.cam_point = cam_point
        self.screen = screen
        self.width = width
        self.height= height
    

    def render(self):

        image = np.zeros((self.height, self.width, 3))
        cam_view = np.array([0., 0., 0.])     # Camera pointing to.
        
        tracer = RayTracer(self.scene, self.light_point, self.color_light, 
                               self.cam_point, self.ambient, self.specular_k)
        
        # Create a progress bar.
        pb_items = self.width / 10
        pb = ProgressBar(pb_items, prefix='Progress:', suffix='Complete', length=50)

        for i, x in enumerate(np.linspace(self.screen[0], self.screen[2], self.width)):

            if i % 10 == 0:
                pb.print_bar(i / 10)

            for j, y in enumerate(np.linspace(self.screen[1], self.screen[3], self.height)):
                cam_view[:2] = (x, y)
                cam_dir = normalize(cam_view - self.cam_point)

                color = tracer.trace_path(self.cam_point, cam_dir)

                image[self.height - j - 1, i, :] = np.clip(color, 0, 1)

        pb.print_bar(pb_items)
        return image
    
    

In [15]:
class SceneBuilder(object):
    
    def add_sphere(center, radius, surface_color, reflection=0):
        
        return Sphere(center=np.array(center), radius=np.array(radius), 
                      surface_color=np.array(surface_color), reflection=.5)
    
    def add_transparent_sphere(center, radius, surface_color, reflection=0):
        
        return Sphere(center=np.array(center), radius=np.array(radius), 
                      surface_color=np.array(surface_color), reflection=.1, 
                      transparency=0.9, ior=1.1)

    
    def add_plane(init_point, normal, reflection=0):
        
        color_0 = 1. * np.ones(3)
        color_1 = 0. * np.ones(3)
        color = lambda point: (color_0 
                        if (int(point[0] * 2) % 2) == (int(point[2] * 2) % 2) 
                               else color_1)
        
        return Plane(init_point=np.array(init_point), normal=np.array(normal), 
                     surface_color=color, diffuse_color=.75, 
                     specular_color=.5, reflection=.25)
    
    def build():
        return [SceneBuilder.add_transparent_sphere([.75, .3, 1.], .8, [0., 0., 1.]),
#                 SceneBuilder.add_sphere([.75, .1, 1.], .8, [0., 0., 1.]),
                SceneBuilder.add_sphere([-.75, .3, 2.25], .8, [.8, .223, .2]),
                SceneBuilder.add_sphere([-2.75, .3, 3.5], .8, [1., .472, .184]),
                SceneBuilder.add_plane([0., -.5, 0.], [0., 1., 0.]),
               ]

In [16]:
WIDTH = 400
HEIGHT = 300
DEPTH_MAX = 5

# Screen coordinates: x0, y0, x1, y1.
ASPECT_RATIO = float(WIDTH) / HEIGHT
SCREEN = (-1., -1. / ASPECT_RATIO + .25, 1., 1. / ASPECT_RATIO + .25)

render = Render(scene=SceneBuilder.build(), 
                light_point=np.array([5., 5., -10.]),
                color_light=np.ones(3),
                specular_k=50,
                ambient=.05,
                cam_point=np.array([0., 0.35, -1.]),
                screen=SCREEN,
                width=WIDTH,
                height=HEIGHT)

image = render.render()
plt.imsave('fig.png', image)

Progress: |██████████████████████████████████████████████████| 100.0% Complete


In [9]:
def id_gen():
    id_ = 0
    while True:
        yield id_
        id_ += 1

get_id = id_gen()

print(next(id_gen()))
print(next(get_id))
print(next(get_id))
print(next(get_id))
print(next(get_id))

0
0
1
2
3


In [12]:
# class Singleton(type):
#     """
#     Metaclass for all singletons in the system (e.g. Kernel).
#     The example code is taken from
#     http://python-3-patterns-idioms-test.readthedocs.io/en/latest/Metaprogramming.html
#     """
#     instance = None

#     def __init__(cls, name, bases, attrs):
#         super().__init__(name, bases, attrs)

#     def __call__(cls, *args, **kw):
#         if not cls.instance:
#             cls.instance = super(Singleton, cls).__call__(*args, **kw)
#         return cls.instance
    
# class Dispatcher(object, metaclass=Singleton):

In [13]:
class Id(object):

    class __Id:
        
        def __init__(self):
            self.generator = self.id_gen()
        
        def id_gen(self):
            id_ = 0
            while True:
                yield id_
                id_ += 1

        def get(self):
            return next(self.generator)
        
        def reset(self):
            self.generator = self.id_gen()

    instance = None

    def __new__(cls):
        if not Id.instance:
            Id.instance = Id.__Id()
        return Id.instance

    def __getattr__(self, name):
        return getattr(self.instance, name)

    def __setattr__(self, name):
        return setattr(self.instance, name)



    
print(Id().get())
print(Id().get())
print(Id().get())
    
ID = Id()
Id().reset()

print(ID.get())
print(ID.get())
print(ID.get())
print(ID.get())
print(ID.get())

ID = Id()

print(ID.get())
print(ID.get())
print(ID.get())
print(ID.get())
print(ID.get())


0
1
2
0
1
2
3
4
5
6
7
8
9


In [37]:
class Ray(object):
    
    class Type(Enum):
        CAMERA = 0
        REFLECTED = 1
        REFRACTED = 2
    
    def __init__(self, start, end, direction=None, length=-1., attenuation=1., cum_att=1., ior=1., type_=Type.CAMERA):
        """
        Creates a new instance of ``Ray``.
        @params:
            ior  : an index of refraction of the media
        """
        self.id = Id().get()
        self.type = type_
        
        # geometrical properties
        self.start = start
        self.end = end
        self.direction = direction if direction is not None else end - start
        self.length = length if length != -1. else la.norm(self.direction)
        
        # Propagation parameters including transmission media change
        self.attenuation = attenuation
        self.cum_att = cum_att * attenuation
        self.ior = ior
    
    def __str__(self):
        string = '<{}({}) start={} end={} dir={} len={:.4}; att={:.4}({:.4}) ior={:.4}>'
        return string.format(self.type.name, self.id, self.start, self.end, 
                             self.direction, self.length, self.attenuation,
                             self.cum_att, self.ior)


ray = Ray(np.array([0,0,0]), np.array([1,1,1]), attenuation=0.5, cum_att=0.9, ior=1.2, type_=Ray.Type.REFLECTED)
print(ray)

<REFLECTED(32) start=[0 0 0] end=[1 1 1] dir=[1 1 1] len=1.732; att=0.5(0.45) ior=1.2>


In [34]:
# class RayTracer(object):
    
#     def __init__(self, scene, light_point, color_light, cam_point, ambient, specular_k):
#         self.scene = scene
#         self.light_point = light_point
#         self.color_light = color_light
#         self.cam_point = cam_point
#         self.ambient = ambient
#         self.specular_k = specular_k

        
#     def trace_ray(self, ray_start, ray_dir):

#         # Find the nearest intersection point with the scene.
#         t = np.inf
#         for s in self.scene:
#             t_i = s.intersect(ray_start, ray_dir)
#             if t_i < t:
#                 t, shape = t_i, s

#         # Return None if the ray does not intersect any object.
#         if t == np.inf:
#             return

#         # Find the intersection point on the object and properties of the object.
#         hit_point = ray_start + ray_dir * t

#         normal = shape.get_normal(hit_point)
#         color = shape.get_color(hit_point)

#         to_light = normalize(self.light_point - hit_point)
#         to_cam = normalize(self.cam_point - hit_point)

#         # Shadow: find if the point is shadowed or not.
#         bias = normal * .0001
#         l = [s.intersect(hit_point + bias, to_light) for s in self.scene if s != shape]
#         if l and min(l) < np.inf:
#             return

#         # Start computing the color.
#         ray_color = self.ambient

#         # Lambert shading (diffuse).
#         ray_color += shape.diffuse_color * max(np.dot(normal, to_light), 0) * color

#         # Blinn-Phong shading (specular).
#         ray_color += shape.specular_color * (max(np.dot(normal, normalize(to_light + to_cam)), 0) 
#                                              ** self.specular_k * self.color_light)

#         return shape, hit_point, normal, ray_color

    
#     def trace_path(self, ray_start, ray_dir):
    
#         color = np.zeros(3)
#         reflection_att = 1.

#         for depth in range(DEPTH_MAX):
#             traced = self.trace_ray(ray_start, ray_dir)
#             if not traced:
#                 break

#             shape, hit_point, normal, ray_color = traced

#             bias = normal * .0001

#             # Reflection: create a new ray, compute current color and reflected ray attenuation.
#             ray_start = hit_point + bias
#             ray_dir   = normalize(ray_dir - 2 * np.dot(ray_dir, normal) * normal)

#             color += reflection_att * ray_color
#             reflection_att *= shape.reflection

#         return color

In [4]:
# class RayTracer(object):
    
#     def __init__(self, scene, light_point, color_light, cam_point, ambient, specular_k):
#         self.scene = scene
#         self.light_point = light_point
#         self.color_light = color_light
#         self.cam_point = cam_point
#         self.ambient = ambient
#         self.specular_k = specular_k

        
#     def trace_ray(self, ray_start, ray_dir):

#         # Find the nearest intersection point with the scene.
#         t = np.inf
#         for s in self.scene:
#             t_i = s.intersect(ray_start, ray_dir)
#             if t_i < t:
#                 t, shape = t_i, s

#         # Return None if the ray does not intersect any object.
#         if t == np.inf:
#             return

#         # Find the intersection point on the object and properties of the object.
#         hit_point = ray_start + ray_dir * t

#         normal = shape.get_normal(hit_point)
#         color = shape.get_color(hit_point)

#         to_light = normalize(self.light_point - hit_point)
#         to_cam = normalize(self.cam_point - hit_point)

#         # Shadow: find if the point is shadowed or not.
#         bias = normal * .0001
#         l = [s.intersect(hit_point + bias, to_light) for s in self.scene if s != shape]
#         if l and min(l) < np.inf:
#             return

#         # Start computing the color.
#         ray_color = self.ambient

#         # Lambert shading (diffuse).
#         ray_color += shape.diffuse_color * max(np.dot(normal, to_light), 0) * color

#         # Blinn-Phong shading (specular).
#         ray_color += shape.specular_color * (max(np.dot(normal, normalize(to_light + to_cam)), 0) 
#                                              ** self.specular_k * self.color_light)

#         return shape, hit_point, normal, ray_color

    
#     def trace_path(self, ray_start, ray_dir):
    
#         color = np.zeros(3)
#         attenuation = 1.
        
#         # Define a list of rays to be traced to enable tracing both reflecting and refracting rays.
#         # Thus tracing procedures perform width-first traversal.
#         # The element of the list:
#         #    (``ray_start``, ``ray_direction``, ``depth``, ``inside``, ``type``)
#         # 
#         # where inside tracks the position of the ray (inside or outside of the object) to compute
#         # a refracted ray direction if the ray exists (transparency is not zero) as it affects
#         # index of refractivity (ior) computation.``type`` parameter is a type of the ray and it shows
#         # whether ray is reflected or refracted.

#         rays_to_trace = [(ray_start, ray_dir, 0, False, RayType.CAMERA)]

#         while rays_to_trace:

#             ray_start, ray_dir, depth, inside, ray_type = rays_to_trace.pop(0)
#             if depth == DEPTH_MAX:
#                 continue

#             traced = self.trace_ray(ray_start, ray_dir)
#             if not traced:
#                 continue

#             shape, hit_point, normal, ray_color = traced
#             bias = normal * .0001
#             neg_dn = -np.dot(ray_dir, normal)

#             # Reflection: create a new ray and append it to the rays list to trace further.
#             reflected_start = hit_point + bias
#             reflected_dir   = ray_dir + 2 * neg_dn * normal
            
#             # Parameter ``inside`` does not change as the ray stays at the same media.
#             rays_to_trace.append((reflected_start, reflected_dir, depth + 1, inside, RayType.REFLECTED))

#             # Refraction: create a new ray and append it to the rays list to trace further.
#             if shape.transparency:

#                 eta = shape.ior if inside else 1 / shape.ior
#                 k = 1 - eta ** 2 * (1 - neg_dn ** 2)

#                 if k >= 0: # total reflection under refraction into the environment with less ior

#                     cos_refracted = np.sqrt(k)
#                     refracted_dir = ray_dir * eta + normal * (eta * neg_dn - cos_refracted)
#                     refracted_start = hit_point - bias

#                     # Parameter ``inside`` changes as the ray enter new media.
#                     rays_to_trace.append((refracted_start, refracted_dir, depth + 1, not inside, RayType.REFRACTED))

#             # Compute current color and reflected ray attenuation.
#             m_att = 1
#             if ray_type is RayType.REFLECTED:
#                 m_att = 1
#             elif ray_type is RayType.REFRACTED:
#                 m_att = 1
                
#             color += attenuation * ray_color
#             attenuation *= shape.reflection # TODO: refraction and reflection color mixing

#         return color

SyntaxError: invalid syntax (<ipython-input-4-d1ea39510dc6>, line 76)